In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

dataLabels = pd.read_csv('Image_classification_data/data_labels_mainData.csv')

img_size = (27, 27)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


data_dir = 'Image_classification_data/patch_images'  


train_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName', 
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_dataframe(
    dataframe=dataLabels,
    directory='Image_classification_data/patch_images',
    x_col='ImageName',  
    y_col='cellTypeName',  
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 7917 validated image filenames belonging to 4 classes.
Found 1979 validated image filenames belonging to 4 classes.


In [32]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(27, 27, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # Use softmax for multi-class classification
])

/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['categorical_accuracy']
)

model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10


/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/home/leo/cancerAI/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:666: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


248/248 ━━━━━━━━━━━━━━━━━━━━ 9s 26ms/step - categorical_accuracy: 0.4386 - loss: 1.2151 - val_categorical_accuracy: 0.6842 - val_loss: 0.8354
Epoch 2/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - categorical_accuracy: 0.6573 - loss: 0.8188 - val_categorical_accuracy: 0.7256 - val_loss: 0.7540
Epoch 3/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - categorical_accuracy: 0.7006 - loss: 0.7382 - val_categorical_accuracy: 0.6205 - val_loss: 0.9587
Epoch 4/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - categorical_accuracy: 0.7144 - loss: 0.7091 - val_categorical_accuracy: 0.7019 - val_loss: 0.8055
Epoch 5/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - categorical_accuracy: 0.7159 - loss: 0.6958 - val_categorical_accuracy: 0.6913 - val_loss: 0.7951
Epoch 6/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - categorical_accuracy: 0.7255 - loss: 0.6873 - val_categorical_accuracy: 0.7388 - val_loss: 0.7282
Epoch 7/10
248/248 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - categorical_accuracy: 0.7324 - loss: 0

In [34]:
model.evaluate(val_data)
model.save('cancerAI.keras')

62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - categorical_accuracy: 0.7364 - loss: 0.7711
